In [1]:
import pandas as pd
import numpy as np
from scipy import stats

# Node class

In [2]:
# node class to store all nodes and leafs in the tree
class Node:
    def __init__(self, feature = None, categories = [], threshold = None, value= None, children = [], label_mode = None):
        self.feature = feature
        self.threshold = threshold
        self.categories = categories
        self.value = value
        self.children = children
        self.label_mode = label_mode

# split criterium

In [3]:
#computes the squared error of each split, for identifying the MSE
def squared_error (y):
    estimate = np.mean(y)
    return [(actual-estimate)**2 for actual in y]

#calculates and returns the entropy of a certain split of a feature given the class frequencies
def entropy (y):
    class_counts = np.bincount(y)
    class_frequency = class_counts/len(y)
    class_frequency = [freq for freq in class_frequency if freq>0]
    return -np.sum((class_frequency * np.log2(class_frequency)))

# stop criterium

In [4]:
#stop criteria for regression 
def stop_regression (current_depth, max_depth, early_stop, mse):
    if (current_depth==max_depth or mse <= early_stop):
        return True
    else:
        return False   
#stop criteria for classification    
def stop (y, current_depth, max_depth, min_split, labels):
    if ( current_depth==max_depth or len(labels)==1):#len(y) < min_split or
        return True
    else:
        return False

# Split by data type and model type

In [5]:
def categorical_splitting_classification (y, feature_vals):
    #calculates the entropy at the root of this subtree
    root_entropy = entropy(y)
    #get the list of the unique categories for the feature and append them
    unique_feature_vals = np.unique(feature_vals)
    
    weighted_gain = 0
    y_args = []
    entropies = []
    #iterate through each categorical feature value
    for feature_val in unique_feature_vals:
        #get the indices for all the samples with this value
        feature_args = np.argwhere(feature_vals == feature_val).flatten()
        y_args.append(feature_args)
        #calculate the frequency of this feature value
        feature_frequency = len(feature_args)/len(feature_vals)
        #get the y values for each index of this feature value
        feature_val_y_vals = y[feature_args]
        #calculates the entropy for this feature value and multiplies it by its frequency to weight it
        entropies.append(feature_frequency*entropy(feature_val_y_vals))
        #add to weighted gain list for gain ratio
        weighted_gain += (feature_frequency*np.log2(feature_frequency))
    #append the list of the indices for each feature value for splitting in the future
    #feature_val_y_args_list.append(y_args)
    #calculate the feature's gain and add it to the list
    feature_gain = (root_entropy-np.sum(entropies))/(-weighted_gain)
    #feature_gains.append(feature_gain)
    #categories.append(unique_feature_vals)
    #threshold.append(None)
    metric_dict = {
        'y_args': y_args,
        'gain': feature_gain,
        'categories': unique_feature_vals,
        'threshold': None
    }
    return metric_dict
def numerical_splitting_classification (y, feature_vals):
    #calculates the entropy at the root of this subtree
    root_entropy = entropy(y)
    #print(list(feature_vals))
    #get the list of the unique categories for the feature and append them
    unique_feature_vals = np.unique(feature_vals)
    #we only need the values for the feature with the optimal threshold, so we use placeholders
    best_gain = -1
    best_y_args = None
    best_threshold = None
    best_category = None
    #iterate through each unique feature value and use it as the threshold
    for feature_val in unique_feature_vals:
        weighted_gain = 0
        category = None
        y_args = [np.argwhere(feature_vals < feature_val).flatten(), np.argwhere(feature_vals >= feature_val).flatten()]
        threshold = feature_val
        #split the data by the threshold
        left = y[y_args[0]]
        right = y[y_args[1]]
        #temp list for entropies of each side
        entropies = []
        #iterate through both sides and calculate the entropies for each
        for side in [left, right]:
            feature_frequency = len(side)/len(feature_vals)
            entropies.append(feature_frequency*entropy(side))
            weighted_gain+= (feature_frequency*np.log2(feature_frequency))
        #calculate the gain for this threshold's split
        gain = root_entropy-np.sum(entropies)/(-weighted_gain)
        #if this gain is higher than our current best, replace it with this one
        if gain > best_gain:
            best_gain = gain
            best_y_args = y_args
            best_threshold = threshold
            best_category = category
    metric_dict = {
        'y_args': best_y_args,
        'gain': best_gain,
        'categories': best_category,
        'threshold': best_threshold
    }
    return metric_dict 

def splitting_regression(y, feature_vals):
    unique_feature_vals = np.unique(feature_vals)
    best_mse = 99999999999
    best_y_args = None
    best_threshold = None
    for feature_val in unique_feature_vals:
        y_args = [np.argwhere(feature_vals < feature_val).flatten(), np.argwhere(feature_vals >= feature_val).flatten()]
        #split the data by the threshold
        left = y[y_args[0]]
        right = y[y_args[1]]
        if((len(left)!=0 and len(right)!=0)):
            split_mse = np.mean(squared_error(left) + squared_error(right))
            if best_mse == 99999999999 or split_mse<best_mse:
                best_mse = split_mse
                best_y_args = y_args
                best_threshold = feature_val
    metric_dict = {
        'y_args': best_y_args,
        'mse': best_mse,
        'threshold': best_threshold
    }
    return metric_dict

# execute splits for fitting tree

In [6]:
#this function is used to grow the regression tree. it runs recursively on the samples and as we split, the samples split with it
#it returns the root of the tree
def execute_regression_split (x, y, max_depth, depth, early_stop, mse):
    #return a leaf if we should stop 
    if (stop_regression(depth, max_depth, early_stop, mse)):
        leaf_val = np.median(y)
        return Node(value = leaf_val)
    n_samples, feature_count = x.shape
    feature_mses = []
    feature_val_y_args_list = []
    threshold = []
    #iterate through each feature in the samples
    for feature_index in range(0, feature_count):
        feature_vals = x[:, feature_index]
        if (len(np.unique(feature_vals))!=1):
            feature_metrics = splitting_regression(y, feature_vals)
            #append our optimal threshold split to the main list 
            feature_val_y_args_list.append(feature_metrics['y_args'])
            feature_mses.append(feature_metrics['mse'])
            threshold.append(feature_metrics['threshold'])
        else:
            feature_val_y_args_list.append([])
            feature_mses.append(999999999)
            threshold.append(0)
    
    best_feature_split = np.argmin(feature_mses)
    best_feature_val_y_args = feature_val_y_args_list[best_feature_split]
    best_threshold = threshold[best_feature_split]
    if (len(best_feature_val_y_args)==0):
        leaf_val = np.median(y)
        return Node(value = leaf_val)
    depth+=1
    #recursively call this function on the two sides of the optimal split
    children = [execute_regression_split(x[feature_args, :], y[feature_args], max_depth, depth, early_stop, feature_mses[best_feature_split]) for feature_args in best_feature_val_y_args]
    return Node(feature = best_feature_split, children = children, threshold = best_threshold)

#this function is used to grow the classification tree. it runs recursively on the samples and as we split, the samples split with it
#it returns the root of the tree
def execute_split (x, y, feature_count, depth):
    labels,counts = np.unique(y,return_counts = True)
    label_mode = labels[np.argmax(counts)]
    #returns a leaf node if we have reached a stopping point
    if (stop(y, depth, max_depth = 10, min_split = 2, labels = labels)):
        leaf_val = labels[np.argmax(counts)]
        return Node(value = leaf_val)
    
    #place holders for the gains of each feature, the list of each value for the feature, and the indices of the y vals for each feature
    feature_gains = []
    feature_val_y_args_list = []
    categories = []
    threshold = []
    #iterate through each feature in the samples
    for feature_index in range(0, feature_count):
        feature_vals = x[:, feature_index]
        feature_val_entropies = []
        #if else to treat categorical and numerical data differently
        if all(isinstance(val, str) for val in feature_vals):
            feature_metrics = categorical_splitting_classification(y, feature_vals)
        #numeric
        else:
            feature_metrics = numerical_splitting_classification(y, feature_vals)
        #append our optimal threshold split to the main list 
        feature_val_y_args_list.append(feature_metrics['y_args'])
        feature_gains.append(feature_metrics['gain'])
        categories.append(feature_metrics['categories'])
        threshold.append(feature_metrics['threshold'])
    #identify best split
    best_feature_split = np.argmax(feature_gains)
    best_feature_val_y_args = feature_val_y_args_list[best_feature_split]
    best_categories = categories[best_feature_split]
    best_threshold = threshold[best_feature_split]
    depth+=1
    #recursively call this function on the x children of the optimal split
    children = [execute_split(x[feature_args, :], y[feature_args], feature_count, depth) for feature_args in best_feature_val_y_args]

    return Node(feature = best_feature_split, children = children, threshold = best_threshold, categories = best_categories, label_mode = label_mode)



    




# fit methods

In [7]:
#fits the decision tree regressor, just runs the above function and returns the root
def fit_decision_tree_regressor (x, y, max_depth = None, p_features = None, early_stop = None):
    root = execute_regression_split(x, y, max_depth = 10, depth=0, early_stop=early_stop, mse=999999999)
    return root
#fits the decision tree classifier, just runs the above function and returns the root
def fit_decision_tree (x, y, max_depth = None, p_features = None, min_split = None):
    n_samples, n_features = x.shape
    root = execute_split(x, y, n_features, depth=0)
    return root

# predict methods

In [8]:
#iterates through each sample and runs it through the predict split method below
def predict_regression (x, root):
    predictions = []
    for sample in x:
        predictions.append(predict_split_regression(sample, root))
    return predictions

#recursive method that traverses through the fitted regression tree 
def predict_split_regression (sample, root):
    if str(root.value).replace('.','').isnumeric():
        return root.value
    else:
        current_feature = sample[root.feature]
        children = root.children
        if current_feature< root.threshold:
            return predict_split_regression(sample, children[0])
        else:
            return predict_split_regression(sample, children[1])

#iterates through each sample and runs it through the predict split method below
def predict_clf (x, root):
    predictions = []
    for sample in x:
        predictions.append(predict_split_clf(sample, root))
    return predictions

#recursive method that traverses through the fitted regression tree 
def predict_split_clf (sample, root):
    if str(root.value).isnumeric():
        return root.value
    else:
        current_feature = sample[root.feature]
        children = root.children
        if isinstance(current_feature, str):
            for index, category in enumerate(root.categories):
                if current_feature == category:
                    return predict_split_clf(sample, children[index])
            #categorical labels can potentially be dropped when fitting the tree as we filter the subsets
            #if this happens, we should return the mode label at that subroot
            return root.label_mode
        else:
            if current_feature< root.threshold:
                return predict_split_clf(sample, children[0])
            else:
                return predict_split_clf(sample, children[1])

# Reduced error pruning

In [9]:
#this takes in the fitted tree and validation set and prunes the classification tree
#it does this by testing the original accuracy vs the pruined accuracy, if the pruned accuracy is higher, keep the prune
def prune_tree(root, current_node, x, y, x_validation, y_validation):
    if current_node.value == None:
        current_feature = current_node.feature
        feature_vals = x[:, current_feature]
        unique_feature_vals = np.unique(feature_vals)
        #current accuracy
        y_predict = predict_clf (x_validation, root)
        results = [y_predict[result_index]==actual for result_index, actual in enumerate(y_validation)]
        current_accuracy = results.count(True)/len(results)
        #compute pruined accuracy
        node_threshold = current_node.threshold
        node_categories = current_node.categories
        labels,counts = np.unique(y,return_counts = True)
        leaf_val = labels[np.argmax(counts)]
        current_node.value = leaf_val
        pruned_precict = predict_clf(x_validation, root)
        pruned_results = [pruned_precict[result_index]==actual for result_index, actual in enumerate(y_validation)]
        pruned_accuracy = pruned_results.count(True)/len(pruned_results)
        
        if pruned_accuracy >= current_accuracy:
            return root
        else:
            #set the value of the node back to None to make it a node again
            current_node.value = None
            if isinstance(feature_vals[0], str):
                feature_args = [np.argwhere(feature_vals == node_category).flatten() for node_category in node_categories ]
            else:
                feature_args = [np.argwhere(feature_vals < node_threshold).flatten(), np.argwhere(feature_vals >= node_threshold).flatten()]
            #recursively call the method on each child
            for index, child in enumerate(current_node.children):
                split_arg = feature_args[index]
                if len(feature_args[index])>0:
                    root = prune_tree(root, child, x[split_arg, :], y[split_arg], x_validation, y_validation)
    return root

In [15]:
#split the data into 5 different sets and return each set, also allows for the creation of a validation set
def validation_sets(data, classification, optimize = False):
    sets = []
    sample_size = len(data)
    if optimize:
        if classification:
            validation_set = data.groupby('target').apply(lambda x: x.sample(frac = 0.1)).reset_index(level=0, drop=True)
        else:
            validation_set = data.sample(n=int(sample_size*0.1))
        sample_size = len(data) - len(validation_set)
        data = data.drop(validation_set.index)
    else:
        validation_set = None
    if classification:
        class_splits = []
        for class_val in np.unique(data['target']):
            df_class = data[data['target'] == class_val]
            dfs_class = np.array_split(df_class, 5)
            class_splits.append(dfs_class)
    for index in range(0, 5):
        #for classification, make sure each class has equal representation for each set as they do in the full dataset
        if classification:
            sample_set_list = []
            for class_index in range(len(class_splits)):
                sample_set_list.append(class_splits[class_index][index])
            sample_set = pd.concat(sample_set_list)
        #we cant do the above for regression, so just split the data normally 
        else:
            sample_set = data.sample(n=int(sample_size*0.2))
            data = data.drop(sample_set.index)
        sets.append(sample_set.reset_index().drop(columns= 'index'))
    return sets, validation_set

#this is the shell for all our model. it takes each set and tests it against the rest of the sets and returns some metrics we need 
#to evaluate our model. it also allows for pruning for classification, and optimizing an MSE value for early stopping for regression
#these require validation sets.
def k_fold_cross_validation(sets, classification = True, validation_set = None, pruning = False,  mse_list = [0.2]):
    scores = []
    #iterate through each set as the test set and calculate metrics
    if pruning or len(mse_list) > 1:
        y_validation = np.array(validation_set['target'])
        x_validation = validation_set.reset_index().drop(columns= ['index','target']).values
    if len(mse_list)>1:
        training_set = pd.concat(sets)
        y_train = np.array(training_set['target'])
        #drop target so its not considered a feature
        x_train = training_set.reset_index().drop(columns= ['index','target']).values
        best_result = None
        best_mse = None
        for mse in mse_list:
            root = fit_decision_tree_regressor(x_train, y_train, max_depth = None, p_features = None, early_stop = mse)
            #runs the data through our regression model
            y_predict_validation = predict_regression(x_validation, root)
            result_rsquared = 1- (((y_validation-y_predict_validation)**2).sum(axis=0)/((y_validation-np.average(y_validation, axis=0,))**2).sum(axis=0))
            if best_result == None or result_rsquared >= best_result:
                best_mse= mse
                best_result = result_rsquared
    else:
        best_mse = mse_list[0]
    
    for index in range(0, len(sets)):
        test_set = sets[index]
        #concats the rest of the sets for training
        training_set = pd.concat([t_set for (set_index, t_set) in enumerate(sets) if set_index!=index])
        y_train = np.array(training_set['target'])
        #drop target so its not considered a feature
        x_train = training_set.reset_index().drop(columns= ['index','target']).values

        y_test = np.array(test_set['target'])
        x_test = test_set.reset_index().drop(columns= ['index','target']).values
        
        #here we either choose to treat the data as classification or regression
        if classification:
            root = fit_decision_tree (x_train, y_train, max_depth = None, p_features = None, min_split = None)
            if pruning == True:
                root = prune_tree(root, root, x_validation, y_validation, x_validation, y_validation,)
            y_predict = predict_clf (x_test, root)
            results = [y_predict[result_index]==actual for result_index, actual in enumerate(y_test)]
            scores.append(results.count(True)/len(results))
        else:
            
            root = fit_decision_tree_regressor(x_train, y_train, max_depth = None, p_features = None, early_stop = best_mse)
            y_predict = predict_regression(x_test, root)
            avg_deviation = np.mean([(np.abs(y_predict[i]-y_test[i]))/(y_test[i]) for i in range(0, len(y_test))])
            result_mse = np.square(y_test - y_predict).mean()
            rsquared = 1- (((y_test-y_predict)**2).sum(axis=0)/((y_test-np.average(y_test, axis=0,))**2).sum(axis=0))
            scores.append([best_mse, result_mse, rsquared, avg_deviation])
    if classification:
        average_results = np.mean(scores)
    else:
        average_results = []
        for index in range(len(scores)-1):
            average_results.append(np.mean([scores[0][index], scores[1][index],scores[2][index],scores[3][index]]))
    return average_results

# Image Segmentation

In [11]:
segmentation = pd.read_csv('segmentation.data')
segmentation = segmentation.reset_index().rename(columns = {'index': 'target'})
segmentation = segmentation.drop(columns = 'REGION-PIXEL-COUNT')
labels = np.unique(segmentation['target'])
segmentation['target'] = [np.where(labels == label)[0][0] for label in segmentation['target']]

In [16]:
sets, validation_set = validation_sets(segmentation, classification = True, optimize = False)
k_fold_cross_validation(sets, classification= True, validation_set = None, pruning = False)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in log2
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in double_scalars


0.8714285714285713

In [20]:
sets, validation_set = validation_sets(segmentation, classification = True, optimize = True)
k_fold_cross_validation(sets, classification= True, validation_set = validation_set, pruning = True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in log2
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in double_scalars


0.781904761904762

# Abalone

In [21]:
abalone = pd.read_csv('abalone.data', header = None)
abalone.columns = ['sex', 'length', 'diameter', 'height', 'whole_weight', 'shucked_weight', 'viscera_weight', 'shell_weight', 'target']

In [22]:
sets, validation_set = validation_sets(abalone, classification = True, optimize = False)
k_fold_cross_validation(sets, classification= True, validation_set = None, pruning = False)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in log2
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in double_scalars
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


0.24204419336942898

In [23]:
sets, validation_set = validation_sets(abalone, classification = True, optimize = True)
k_fold_cross_validation(sets, classification= True, validation_set = validation_set, pruning = True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in log2
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in double_scalars
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


0.24259453644033124

# Cars

In [24]:
cars = pd.read_csv('car.data', header = None)
cars.columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'target']

In [25]:
cars['doors'] = pd.to_numeric(cars['doors'].replace('5more', 5))
cars['persons'] = pd.to_numeric(cars['persons'].replace('more', 6))
cars['target'] = pd.to_numeric(cars['target'].replace('unacc', 0).replace('acc', 1).replace('good', 2).replace('vgood', 3))

In [26]:
sets, validation_set = validation_sets(cars, classification = True, optimize = False)
k_fold_cross_validation(sets, classification= True, validation_set = None, pruning = False)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in log2
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in double_scalars
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


0.7002352466729399

In [27]:
sets, validation_set = validation_sets(cars, classification = True, optimize = True)
k_fold_cross_validation(sets, classification= True, validation_set = validation_set, pruning = True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in log2
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in double_scalars
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


0.7005186185037007

# Machines

In [28]:
machines = pd.read_csv('machine.data', header=None)
machines.columns = ['vendor', 'model', 'myct', 'mmin', 'mmax', 'cach', 'chmin', 'chmax', 'target', 'erp']
machines = machines.drop(columns = ['model', 'erp'])
machines = machines.drop(columns = 'vendor') #.join(pd.get_dummies(machines['vendor']))

In [29]:
sets, validation_set = validation_sets(machines, classification = False)
k_fold_cross_validation(sets, classification = False, mse_list = [1000])

[1000.0, 4704.387195121952, 0.8138258215609964, 0.5368478179743618]

# Forest fires

In [30]:
forest_fires = pd.read_csv('forestfires.data')
forest_fires = forest_fires.rename(columns = {'area': 'target'})
forest_fires['target'] = forest_fires['target']#.apply(lambda x: np.log(x+1))

In [31]:
#change month and days to #s
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
days = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
forest_fires['day'] = forest_fires['day'].apply(lambda x: days.index(x) if str(x).isalpha() else x)
forest_fires['month'] = forest_fires['month'].apply(lambda x: months.index(x))

In [32]:
sets, validation_set = validation_sets(forest_fires, classification = False)
k_fold_cross_validation(sets, classification = False, mse_list = [4000])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:84: RuntimeWarning: divide by zero encountered in double_scalars
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in double_scalars


[4000.0, 5049.758879733008, -0.17698036753428564, nan]

# Wine quality - red

In [35]:
wine_quality_red = pd.read_csv('winequality-red.csv', delimiter = ';').rename(columns = {'quality': 'target'})

In [36]:
sets, validation_set = validation_sets(wine_quality_red, classification = False)
k_fold_cross_validation(sets, classification = False, mse_list = [0.4])

[0.4, 0.6130485893416928, 0.07839679272834488, 0.08867181668905808]

# Wine quality - white

In [37]:
wine_quality_white = pd.read_csv('winequality-white.csv', delimiter = ';').rename(columns = {'quality': 'target'})

In [ ]:
sets, validation_set = validation_sets(wine_quality_white, classification = False)
k_fold_cross_validation(sets, classification = False, mse_list = [0.4])